#### Produce cleaning procedure locally on a small sample

In [ ]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from pandas.tseries import offsets
from pandas.tseries.offsets import *

DEVELOPE = False

### Step1 download and read data
Save a smaller sample when developing

In [ ]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + "yellow_tripdata_2015-01-06.csv")

In [ ]:
if DEVELOPE:
    df = df.sample(frac=0.1)
else:
    pass
print ("Shape of data\n{}".format(df.shape))

In [ ]:
df.to_csv('yellow_tripdata_2015_short.csv')

#### cells above should be resumed when download original data

In [ ]:
df = pd.read_csv('data/yellow_tripdata_2015_short.csv', index_col=0)
print ("Shape of data\n{}".format(df.shape))
df.head()

In [ ]:
df.columns

In [ ]:
df = df.drop(['VendorID', 'trip_distance',
         'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
         'tip_amount', 'tolls_amount', 'total_amount'], axis=1)
df.dropna(inplace=True)
df.head()

In [ ]:
df.shape

#### borders of NY City
city_long_border = (-74.03, -73.75);
city_lat_border = (40.63, 40.85) 

In [ ]:
df = df[df['pickup_longitude'] <= -73.75]
df = df[df['pickup_longitude'] >= -74.03]
df = df[df['pickup_latitude'] <= 40.85]
df = df[df['pickup_latitude'] >= 40.63]
df = df[df['dropoff_longitude'] <= -73.75]
df = df[df['dropoff_longitude'] >= -74.03]
df = df[df['dropoff_latitude'] <= 40.85]
df = df[df['dropoff_latitude'] >= 40.63]
df.shape

#### extract all the time features out

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y/%m/%d %H:%M:%S')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y/%m/%d %H:%M:%S')

In [ ]:
df.dtypes

In [ ]:
df['year'] = df['tpep_pickup_datetime'].dt.year
df['month'] = df['tpep_pickup_datetime'].dt.month
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['dayofweek'] = df['tpep_pickup_datetime'].dt.weekday_name
df['year_quarter'] = df['tpep_pickup_datetime'].dt.quarter
df['days_in_month'] = df['tpep_pickup_datetime'].dt.days_in_month

In [ ]:
df.head()

In [ ]:
df.columns

#### delete trips beyond reasonable time range

In [ ]:
df['timerange'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [ ]:
df['timerange'] = df['timerange'].dt.seconds

In [ ]:
df['timerange']

In [ ]:
df['timerange'].describe()

In [ ]:
df.head()

In [ ]:
df['timerange'].unique()

In [ ]:
#12 hours: 12*60*60=43200
df = df[df['timerange'] > 10]
df = df[df['timerange'] < 43200]

In [ ]:
df.describe()

#### For the holiday:
#### I tried to extract whether certain day is the day before a holiday, which I expect lots of people will rush to airport. However I fail to use the 'Holidays / Holiday Calendars' class in pandas which contains 'USFederalHolidayCalendar'.

In [ ]:
Thanksgiving = ['2017-11-23', '2016-11-24', '2015-11-26', '2014-11-27', '2013-11-28']
ChristmatsEve = ['2017-12-23', '2016-12-23', '2015-12-23', '2014-12-23', '2013-12-23']

In [ ]:
df['tpep_pickup_datetime'].dt.date